In [1]:
import os

import numpy as np
np.set_printoptions(suppress=True)
import pandas as pd

from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

print("Package is ready.")

Package is ready.


# 2020 Summer

In [2]:
pg_df = []

In [3]:
temp_df = pd.read_csv('./results/2020_S/node_length.csv', index_col='Unnamed: 0')
temp_df.columns = temp_df.columns.astype('int64')

last_day = pd.concat([temp_df.loc['2020-07-07'].dropna(),
                      temp_df.loc['2020-07-08'].dropna(),
                      temp_df.loc['2020-07-09'].dropna()]).reset_index().drop('index', axis=1)
last_day.index = list(last_day.index)

temp_index = temp_df.index
temp_df = pd.concat([temp_df.loc[:'2020-06-17'], last_day.T])
temp_df.index = list(temp_index[:-3]) + ['2020-07-03']
temp_df.index = pd.DatetimeIndex(temp_df.index)

l = []
l_i = []
for _ in range(temp_df.shape[0]):
    ll = temp_df.iloc[_].dropna()
    l.extend(ll.values)
    l_i.extend([temp_df.index[_]]*ll.shape[0])
temp_df = pd.DataFrame(l, index=l_i, columns=['node_len'])

pg_df.append(temp_df)

In [4]:
temp_df = pd.read_csv('./results/2020_S/node_diameter.csv', index_col='Unnamed: 0')
temp_df.columns = temp_df.columns.astype('int64')

last_day = pd.concat([temp_df.loc['2020-07-07'].dropna(),
                      temp_df.loc['2020-07-08'].dropna(),
                      temp_df.loc['2020-07-09'].dropna()]).reset_index().drop('index', axis=1)
last_day.index = list(last_day.index)

temp_index = temp_df.index
temp_df = pd.concat([temp_df.loc[:'2020-06-17'], last_day.T])
temp_df.index = list(temp_index[:-3]) + ['2020-07-03']
temp_df.index = pd.DatetimeIndex(temp_df.index)

l = []
l_i = []
for _ in range(temp_df.shape[0]):
    ll = temp_df.iloc[_].dropna()
    l.extend(ll.values)
    l_i.extend([temp_df.index[_]]*ll.shape[0])
temp_df = pd.DataFrame(l, index=l_i, columns=['node_dia'])

pg_df.append(temp_df)

In [5]:
temp_df = pd.read_csv('./results/2020_S/plant_height.csv', index_col='Unnamed: 0')
temp_df.columns = temp_df.columns.astype('int64')

last_day = pd.concat([temp_df.loc['2020-07-07'].dropna(),
                      temp_df.loc['2020-07-08'].dropna(),
                      temp_df.loc['2020-07-09'].dropna()]).reset_index().drop('index', axis=1)
last_day.index = list(last_day.index)

temp_index = temp_df.index
temp_df = pd.concat([temp_df.loc[:'2020-06-17'], last_day.T])
temp_df.index = list(temp_index[:-3]) + ['2020-07-03']
temp_df.index = pd.DatetimeIndex(temp_df.index)

l = []
l_i = []
for _ in range(temp_df.shape[0]):
    ll = temp_df.iloc[_].dropna()
    l.extend(ll.values)
    l_i.extend([temp_df.index[_]]*ll.shape[0])
temp_df = pd.DataFrame(l, index=l_i, columns=['plant_h'])

pg_df.append(temp_df)

In [6]:
temp_df = pd.read_csv('./results/2020_S/leaf_area.csv', index_col='Unnamed: 0')
temp_df.columns = temp_df.columns.astype('int64')

last_day = pd.concat([temp_df.loc['2020-07-07'].dropna(),
                      temp_df.loc['2020-07-08'].dropna(),
                      temp_df.loc['2020-07-09'].dropna()]).reset_index().drop('index', axis=1)
last_day.index = list(last_day.index)

temp_index = temp_df.index
temp_df = pd.concat([temp_df.loc[:'2020-06-17'], last_day.T])/10000
temp_df.index = list(temp_index[:-3]) + ['2020-07-03']
temp_df.index = pd.DatetimeIndex(temp_df.index)

l = []
l_i = []
for _ in range(temp_df.shape[0]):
    ll = temp_df.iloc[_].dropna()
    l.extend(ll.values)
    l_i.extend([temp_df.index[_]]*ll.shape[0])
temp_df = pd.DataFrame(l, index=l_i, columns=['leaf_area'])

pg_df.append(temp_df)

In [7]:
pg_df = pd.concat(pg_df, axis=1)
pg_df['DAT'] = [27]*3 + [55]*3 + [86]*3 + [112]*4 + [128]*20
pg_df['date'] = pg_df.index
pg_df = pg_df.set_index(['DAT'])

In [8]:
pg_reg_df = []
plot_x = [_ for _ in range(14, 129)]
date_x = pd.date_range('2020-03-11', '2020-07-03', freq='1d')

In [9]:
def sigmoid(x, L, k, x0, b):
    return L / (b + np.exp(-k*(x-x0)))

In [10]:
pg_df.iloc[-18, 0] = np.nan
pg_df.iloc[-2, 1] = np.nan
pg_df.iloc[-6, 1] = np.nan
pg_df.iloc[-21, 0] -= 3
pg_df = pg_df.dropna()

In [11]:
x = pg_df.index
y = pg_df['node_len']
polreg = np.poly1d(np.polyfit(x,y,3))
print(r2_score(y, polreg(x)))

plot_y = polreg(plot_x)
pg_reg_df.append(plot_y)

0.7185346190241424


In [12]:
polreg.c

array([ 0.00002078, -0.00446117,  0.28968234, -1.87530395])

In [13]:
x = pg_df.index
y = pg_df['node_dia']
polreg = np.poly1d(np.polyfit(x,y,3))
print(r2_score(y, polreg(x)))

plot_y = polreg(plot_x)
pg_reg_df.append(plot_y)

0.8822503506478435


In [14]:
polreg.c

array([ 0.00001451, -0.00276196,  0.16731076,  1.47825226])

In [15]:
x = pg_df.index
y = pg_df['plant_h']
popt, pcov = curve_fit(sigmoid, x, y, method='trf')
print(r2_score(y, sigmoid(x, *popt)))

plot_y = sigmoid(plot_x, *popt)
pg_reg_df.append(plot_y)

0.8990470668672836


In [16]:
popt

array([ 5.27039071,  0.08027103, -5.88310457,  0.02641877])

In [17]:
x = pg_df.index
y = pg_df['leaf_area']*1000
popt, pcov = curve_fit(sigmoid, x, y, method='trf')
print(r2_score(y, sigmoid(x, *popt)))

plot_y = sigmoid(plot_x, *popt)
pg_reg_df.append(plot_y/1000)

0.8882709607577453


In [18]:
popt/1000

array([ 0.01730997,  0.00007983, -0.00352976,  0.00001327])

In [19]:
popt

array([17.30997002,  0.07983495, -3.52975992,  0.01326671])

In [20]:
pg_reg_df = pd.DataFrame(pg_reg_df).T
pg_reg_df.columns = ['node_len', 'node_dia', 'plant_h', 'leaf_area']
pg_reg_df.index = date_x

In [21]:
pg_reg_df.to_csv('./results/2020_S/pg_reg_result.csv')

# 2020 Winter

In [22]:
pg_df = []

In [23]:
temp_df = pd.read_csv('./results/2020_W/node_length.csv', index_col='Unnamed: 0')
temp_df.columns = temp_df.columns.astype('int64')

temp_index = temp_df.index
temp_df.index = pd.DatetimeIndex(temp_df.index)

l = []
l_i = []
for _ in range(temp_df.shape[0]):
    ll = temp_df.iloc[_].dropna()
    l.extend(ll.values)
    l_i.extend([temp_df.index[_]]*ll.shape[0])
temp_df = pd.DataFrame(l, index=l_i, columns=['node_len'])

pg_df.append(temp_df)

In [24]:
temp_df = pd.read_csv('./results/2020_W/node_diameter.csv', index_col='Unnamed: 0')
temp_df.columns = temp_df.columns.astype('int64')

temp_index = temp_df.index
temp_df.index = pd.DatetimeIndex(temp_df.index)

l = []
l_i = []
for _ in range(temp_df.shape[0]):
    ll = temp_df.iloc[_].dropna()
    l.extend(ll.values)
    l_i.extend([temp_df.index[_]]*ll.shape[0])
temp_df = pd.DataFrame(l, index=l_i, columns=['node_dia'])

pg_df.append(temp_df)

In [25]:
temp_df = pd.read_csv('./results/2020_W/plant_height.csv', index_col='Unnamed: 0')
temp_df.columns = temp_df.columns.astype('int64')

temp_index = temp_df.index
temp_df.index = pd.DatetimeIndex(temp_df.index)

l = []
l_i = []
for _ in range(temp_df.shape[0]):
    ll = temp_df.iloc[_].dropna()
    l.extend(ll.values)
    l_i.extend([temp_df.index[_]]*ll.shape[0])
temp_df = pd.DataFrame(l, index=l_i, columns=['plant_h'])

pg_df.append(temp_df)

In [26]:
temp_df = pd.read_csv('./results/2020_W/leaf_area.csv', index_col='Unnamed: 0')
temp_index = temp_df.index
temp_df.index = pd.DatetimeIndex(temp_df.index)
pg_df.append(temp_df)

In [27]:
pg_df = pd.concat(pg_df, axis=1)
pg_df['DAT'] = [15]*3 + [50]*6 + [80]*6 + [108]*3 + [153]*15
pg_df['date'] = pg_df.index
pg_df = pg_df.set_index(['DAT'])

In [28]:
pg_reg_df = []
plot_x = [_ for _ in range(14, 155)]
date_x = pd.date_range('2020-09-08', '2021-01-26', freq='1d')

In [29]:
def sigmoid(x, L, k, x0, b):
    return L / (b + np.exp(-k*(x-x0)))

In [30]:
x = pg_df.index
y = pg_df['node_len']
polreg = np.poly1d(np.polyfit(x,y,3))
print(r2_score(y, polreg(x)))

plot_y = polreg(plot_x)
pg_reg_df.append(plot_y)

0.5781547139808865


In [31]:
polreg.c

array([-0.00000984,  0.00246222, -0.14693308,  5.00895717])

In [32]:
x = pg_df.index
y = pg_df['node_dia']
polreg = np.poly1d(np.polyfit(x,y,3))
print(r2_score(y, polreg(x)))

plot_y = polreg(plot_x)
pg_reg_df.append(plot_y)

0.6467750963489755


In [33]:
polreg.c

array([-0.00001001,  0.00253274, -0.1485729 ,  5.79562527])

In [34]:
x = pg_df.index
y = pg_df['plant_h']
popt, pcov = curve_fit(sigmoid, x, y, method='trf')
print(r2_score(y, sigmoid(x, *popt)))

plot_y = sigmoid(plot_x, *popt)
pg_reg_df.append(plot_y)

0.88654144620012


In [35]:
popt

array([15.96215235,  0.05966845,  1.72928464,  0.08482201])

In [36]:
x = pg_df.index
y = pg_df['leaf_area']*1000
popt, pcov = curve_fit(sigmoid, x, y, method='trf')
print(r2_score(y, sigmoid(x, *popt)))

plot_y = sigmoid(plot_x, *popt)
pg_reg_df.append(plot_y/1000)

0.5353888365750572


In [37]:
popt/1000

array([0.00151434, 0.00093515, 0.00100452, 0.        ])

In [38]:
popt

array([1.51434386, 0.93515472, 1.00452332, 0.00000013])

In [39]:
pg_reg_df = pd.DataFrame(pg_reg_df).T
pg_reg_df.columns = ['node_len', 'node_dia', 'plant_h', 'leaf_area']
pg_reg_df.index = date_x

In [40]:
pg_reg_df.to_csv('./results/2020_W/pg_reg_result.csv')